# Clasificación de Imágenes MRI con RNN

Este notebook explora el uso de redes neuronales recurrentes (RNN) para la clasificación de imágenes MRI de tumores cerebrales. Incluye:

1. Preprocesamiento y conversión de imágenes a secuencias
2. Definición y entrenamiento de un modelo RNN
3. Evaluación de resultados

In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Flatten

# Configuración de rutas
# DATASET_DIR = "../total/archive/Training"

Las RNN no son ideales para imágenes, pero este notebook es educativo para explorar su uso en visión computacional.

In [2]:
from google.colab import drive
drive.mount('/content/drive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
from pathlib import Path

BASE_DIR = Path("/content/drive/MyDrive/dataset/")
TRAINING_BASE_DIR = BASE_DIR / "Training"
AUGMENTED_BASE_DIR = BASE_DIR / "Augmented"


# Clasificación de Tumores Cerebrales con RNN (Secuencias de Parches de Imagen)

Este notebook implementa una aproximación experimental para usar RNN en imágenes MRI, convirtiendo cada imagen en una secuencia de parches y clasificando con una red recurrente.

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Dropout, BatchNormalization, TimeDistributed, Flatten
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2

np.random.seed(42)
tf.random.set_seed(42)

gpus = tf.config.experimental.list_physical_devices("GPU")
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

classes = [d.name for d in AUGMENTED_BASE_DIR.iterdir() if d.is_dir()]
num_classes = len(classes)

IMG_SIZE = 512
BATCH_SIZE = 64
EPOCHS = 100
LEARNING_RATE = 0.001
DROPOUT_RATE = 0.5
L2_REG = 0.0001
PATCH_SIZE = 16
PATCHES = (IMG_SIZE // PATCH_SIZE) ** 2
PATCH_DIM = PATCH_SIZE * PATCH_SIZE * 3

In [ ]:
def image_to_sequence(img):
    patches = []
    h, w, c = img.shape
    for i in range(0, h, PATCH_SIZE):
        for j in range(0, w, PATCH_SIZE):
            patch = img[i:i+PATCH_SIZE, j:j+PATCH_SIZE, :].reshape(-1)
            patches.append(patch)
    return np.array(patches)

def generator_with_sequence(generator):
    while True:
        images, labels = next(generator)
        batch_seq = np.array([image_to_sequence(img) for img in images])
        yield batch_seq, labels

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,
    validation_split=0.2,
)

train_generator = train_datagen.flow_from_directory(
    str(AUGMENTED_BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="training",
    shuffle=True,
    color_mode="rgb",
)

val_generator = train_datagen.flow_from_directory(
    str(AUGMENTED_BASE_DIR),
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    subset="validation",
    shuffle=False,
    color_mode="rgb",
)

In [ ]:
def create_brain_tumor_rnn(input_shape=(PATCHES, PATCH_DIM), num_classes=num_classes):
    model = Sequential([
        SimpleRNN(128, input_shape=input_shape, return_sequences=True, kernel_regularizer=l2(L2_REG)),
        BatchNormalization(),
        Dropout(DROPOUT_RATE),
        SimpleRNN(64, return_sequences=False, kernel_regularizer=l2(L2_REG)),
        BatchNormalization(),
        Dropout(DROPOUT_RATE),
        Dense(64, activation="relu", kernel_regularizer=l2(L2_REG)),
        BatchNormalization(),
        Dropout(DROPOUT_RATE),
        Dense(num_classes, activation="softmax")
    ])
    model.compile(optimizer=Adam(learning_rate=LEARNING_RATE), loss="categorical_crossentropy", metrics=["accuracy", tf.keras.metrics.AUC(name="auc")])
    return model

model = create_brain_tumor_rnn(input_shape=(PATCHES, PATCH_DIM), num_classes=4)
model.summary()

callbacks = [
    EarlyStopping(monitor="val_auc", patience=15, mode="max", restore_best_weights=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.2, patience=8),
    ModelCheckpoint(BASE_DIR/"models/best_rnn_model.keras", monitor="val_auc", save_best_only=True, mode="max"),
    TensorBoard(log_dir="./logs_rnn"),
]




In [ ]:
history = model.fit(
    generator_with_sequence(train_generator),
    steps_per_epoch = int(0.7 * train_generator.samples // BATCH_SIZE),
    validation_data=generator_with_sequence(val_generator),
    validation_steps = int(0.7 * val_generator.samples // BATCH_SIZE),
    epochs=EPOCHS,
    callbacks=callbacks,
)


In [ ]:
val_loss, val_acc, val_auc = model.evaluate(generator_with_sequence(val_generator), steps=val_generator.samples // BATCH_SIZE)
print(f"\nPrecisión Validación: {val_acc:.4f}")
print(f"AUC Validación: {val_auc:.4f}")

model.save("brain_tumor_rnn_final.h5")

# Aumentación de Datos para Imágenes MRI

Antes de convertir las imágenes a secuencias para la RNN, es recomendable aplicar aumentación para mejorar la robustez del modelo. A continuación se muestra cómo realizar aumentación usando `ImageDataGenerator` de Keras y visualizar ejemplos.

In [ ]:
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from pathlib import Path

from pathlib import Path
from tqdm import tqdm
import cv2
import albumentations as A

AUG_IMG_SIZE = 64
AUG_BATCH_SIZE = 4

# Crear carpetas de salida si no existen
AUGMENTED_BASE_DIR.mkdir(parents=True, exist_ok=True)


AUG_PER_IMAGE = 10  # Número de aumentaciones por imagen
INPUT_DIR = TRAINING_BASE_DIR
OUTPUT_DIR = AUGMENTED_BASE_DIR

# Definir el pipeline de augmentación (basado en el notebook)
transform = A.Compose(
    [
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.5),
        A.Rotate(limit=30, p=0.5),
        A.RandomBrightnessContrast(
            brightness_limit=0.2, contrast_limit=0.2, p=0.3
        ),
        A.GaussianBlur(blur_limit=(3, 7), p=0.2),
        A.CLAHE(p=0.3),
    ]
)


def augment_dataset(input_dir, output_dir, aug_per_image=AUG_PER_IMAGE):
    input_dir = Path(input_dir)
    output_dir = Path(output_dir)
    output_dir.mkdir(parents=True, exist_ok=True)
    for class_dir in input_dir.iterdir():
        if not class_dir.is_dir():
            continue
        out_class_dir = output_dir / class_dir.name
        out_class_dir.mkdir(parents=True, exist_ok=True)
        images = sorted([f for f in class_dir.glob("*.jpg")])
        for img_path in tqdm(images, desc=f"{class_dir.name}"):
            img = cv2.imread(str(img_path))
            if img is None:
                continue
            # Guardar imagen original
            orig_name = out_class_dir / img_path.name
            cv2.imwrite(str(orig_name), img)
            # Generar aumentaciones
            for i in range(aug_per_image):
                augmented = transform(image=img)
                aug_img = augmented["image"]
                aug_name = (
                    out_class_dir / f"{img_path.stem}_aug{i}{img_path.suffix}"
                )
                cv2.imwrite(str(aug_name), aug_img)


In [ ]:
  augment_dataset(INPUT_DIR, OUTPUT_DIR)


In [ ]:
!ls -p  /content/Augmented/glioma/ | grep -v / | wc -l